In [ ]:
# Imports
import astropy.constants as c
import astropy.units as u
import h5py
h5py.enable_ipython_completer()
import numpy as np
import yaml
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set_context('poster')
sns.set_style('whitegrid')

In [ ]:
# Script parameters
file_parameters='comparison.yml'
displayParameters=True

# Read parameters from yaml file
parameters=yaml.load(open(file_parameters),Loader=yaml.FullLoader)
if displayParameters:
    for item in parameters:
        print("{:20s}: {}".format(item,parameters[item]))

haloFile1=parameters['inputFiles']['haloFile1']
subFile1=parameters['inputFiles']['subhaloFile1']
galFile1=parameters['inputFiles']['galaxyFile1']
haloFile2=parameters['inputFiles']['haloFile2']
subFile2=parameters['inputFiles']['subhaloFile2']
galFile2=parameters['inputFiles']['galaxyFile2']
save_prefix=parameters['save_prefix']
graphMin=parameters['graphs']['graphMin']
graphMax=parameters['graphs']['graphMax']
snapMin=parameters['snapshots']['snapMin']
snapMax=parameters['snapshots']['snapMax']

In [ ]:
# Read in halo table
with h5py.File(haloFile1,'r') as f:
    # Get the data from the snapshot
    halos1=f['Halos'][:]
    
# Read in subhalo table
with h5py.File(subFile1,'r') as f:    
    #Get the data from the snapshot
    subs1=f['Halos'][:]
    
# Read in galaxy table
with h5py.File(galFile1,'r') as f:    
    #Get the data from the snapshot
    gals1=f['Galaxies'][:]

    # Read in halo table
with h5py.File(haloFile2,'r') as f:
    # Get the data from the snapshot
    halos2=f['Halos'][:]
    
# Read in subhalo table
with h5py.File(subFile2,'r') as f:    
    #Get the data from the snapshot
    subs2=f['Halos'][:]
    
# Read in galaxy table
with h5py.File(galFile2,'r') as f:    
    #Get the data from the snapshot
    gals2=f['Galaxies'][:]

In [ ]:
nhalo1=len(halos1)
nhalo2=len(halos2)
assert nhalo1==nhalo2
nsub1=len(subs1)
nsub2=len(subs2)
assert nsub1==nsub2
ngal1=len(gals1)
ngal2=len(gals2)
assert ngal1==ngal2

In [ ]:
# Filter out the halos and subhalos in our selection.
# Might be faster to include np.nonzero() on rhs, but this packages indices in a tuple
mask=(graphMin<=halos1['graph_ID']) & (halos1['graph_ID']<=graphMax) & (snapMin<=halos1['snap_ID']) & (halos1['snap_ID']<=snapMax)
halos1=halos1[mask]
halos2=halos2[mask]
#halos=halos[(graphMin<=halos['graph_ID']) & (halos['graph_ID']<=graphMax) & (snapMin<=halos['snap_ID']) & (halos['snap_ID']<=snapMax)]
mask=(graphMin<=subs1['graph_ID']) & (subs1['graph_ID']<=graphMax) & (snapMin<=subs1['snap_ID']) & (subs1['snap_ID']<=snapMax)
subs1=subs1[mask]
subs2=subs2[mask]
mask=(graphMin<=gals1['graph_ID']) & (gals1['graph_ID']<=graphMax) & (snapMin<=gals1['snap_ID']) & (gals1['snap_ID']<=snapMax) & gals1['b_exists']==True
gals1=gals1[mask]
gals2=gals2[mask]

In [ ]:
for plot in parameters['plots'].items():
    print(plot)
    if plot[1]:
        plotName=plot[0]
        plotProps=parameters[plotName]
        print(plotProps)
        plt.figure(figsize=plotProps['figsize'])
        plt.xscale(plotProps['xscale'])
        plt.yscale(plotProps['yscale'])
        plt.title(plotName)
        plt.xlabel(plotProps['xlabel'])
        plt.ylabel(plotProps['ylabel'])
        if plotName=='coldGasMetallicity':
            xdata=gals1['mass_metals_gas_cold']/gals1['mass_gas_cold']
            ydata=gals2['mass_metals_gas_cold']/gals1['mass_gas_cold']
        elif plotName=='hotGasSubhalo':
            xdata=subs1['mass_gas_hot']
            ydata=subs2['mass_gas_hot']
        elif plotName=='stellarDiscRadius':
            xdata=gals1['radius_stars_disc']
            ydata=gals2['radius_stars_disc']
        elif plotName=='stellarMass':
            xdata=gals1['mass_stars_disc']+gals1['mass_stars_bulge']
            ydata=gals2['mass_stars_disc']+gals2['mass_stars_bulge']
        plt.plot(xdata,ydata,plotProps['markerType'],label=plotProps['label'])
        plt.plot([min(xdata),max(xdata)],[min(xdata),max(xdata)],'k:')
        if plotProps['legend']: plt.legend()
        if plotProps['save_png']: plt.savefig('figs/'+save_prefix+plotProps['name']+'.png',bbox_inches='tight')
        if plotProps['save_pdf']: plt.savefig('figs/'+save_prefix+plotProps['name']+'.pdf',bbox_inches='tight')

In [ ]:
xdata

In [ ]:
ydata